In [21]:
import datetime
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime as dt
import numba as nb
from collections import defaultdict 
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

In [22]:
if not mt5.initialize():
    print(mt5.last_error())
    mt5.shutdown()
else:
    authorized=mt5.login(35737953, password="fdphx7cm", server="MetaQuotes-Demo")
    print(mt5.last_error())

(1, 'Success')


In [23]:
def getData(numBefore, timePeriod, signal, timeDict):
    rates = mt5.copy_rates_from_pos(signal, timeDict[timePeriod], 1, numBefore)       
    ratesDf = pd.DataFrame(rates)
    return ratesDf

def getEMA(closeVals, period):
    return closeVals.ewm(span=period).mean()

def getAllInfo(rates, n1, n2):
    ema_1 = getEMA(rates.close, n1)
    ema_2 = getEMA(rates.close, n2)
    
    outDf = rates.copy()
    outDf[f'ema{n1}'] = ema_1
    outDf[f'ema{n2}'] = ema_2
    
    outDf.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)
    finalOut = outDf.copy()
    
    return finalOut

In [24]:
timeDict = {'15': mt5.TIMEFRAME_M15, '30': mt5.TIMEFRAME_M30, '60': mt5.TIMEFRAME_H1}
timePeriod = '15'

a = getData(50, timePeriod, 'AUDUSD', timeDict)
print(getAllInfo(a, 8, 13))

          time     open     high      low    close      ema8     ema13
0   1607080500  0.74209  0.74220  0.74175  0.74183  0.741830  0.741830
1   1607081400  0.74183  0.74259  0.74182  0.74249  0.742201  0.742185
2   1607082300  0.74249  0.74295  0.74227  0.74267  0.742398  0.742372
3   1607083200  0.74266  0.74267  0.74197  0.74197  0.742248  0.742247
4   1607084100  0.74196  0.74232  0.74158  0.74191  0.742143  0.742158
5   1607085000  0.74191  0.74204  0.74169  0.74192  0.742079  0.742101
6   1607085900  0.74193  0.74253  0.74166  0.74248  0.742187  0.742183
7   1607086800  0.74248  0.74268  0.74207  0.74235  0.742229  0.742217
8   1607087700  0.74235  0.74242  0.74171  0.74186  0.742137  0.742149
9   1607088600  0.74188  0.74229  0.74168  0.74221  0.742155  0.742160
10  1607089500  0.74221  0.74261  0.74192  0.74204  0.742128  0.742139
11  1607090400  0.74203  0.74251  0.74184  0.74250  0.742215  0.742200
12  1607091300  0.74249  0.74278  0.74234  0.74257  0.742297  0.742261
13  16

# Check ema crossover for all signals

In [25]:
# Get the signals required
signals = mt5.symbols_get(group="*GBP*, *JPY*, *EUR*, *AUD*, *GBP*, *CHF*, *NZD*, *CAD*")

sigNames = [sig.name for sig in signals]
print(sigNames)

['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY', 'AUDUSD', 'NZDUSD', 'USDCAD', 'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'CADCHF', 'CADJPY', 'CHFJPY', 'EURAUD', 'EURCAD', 'EURCHF', 'EURCZK', 'EURDKK', 'EURGBP', 'EURHKD', 'EURHUF', 'EURJPY', 'EURNOK', 'EURNZD', 'EURPLN', 'EURRUR', 'EURRUB', 'EURSEK', 'EURTRY', 'EURZAR', 'GBPAUD', 'GBPCHF', 'GBPJPY', 'XAUEUR', 'XAUAUD', 'XAGEUR', 'GBPCAD', 'GBPNOK', 'GBPNZD', 'GBPPLN', 'GBPSEK', 'GBPSGD', 'GBPZAR', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDSGD', 'SGDJPY', 'BTCEUR', 'BTCJPY', 'LTCEUR', 'LTCJPY', 'EURMXN', 'GBPMXN', 'CADMXN', 'CHFMXN', 'MXNJPY', 'NZDMXN']


In [26]:
# sigNames = sigNames[0:3]
# sigNames

In [41]:
# Set up the variables required
barsAfter = [3, 5, 10]
resultDict = {}

## Get the values and compare with real data to validate
timeDict = {'15': mt5.TIMEFRAME_M15, '30': mt5.TIMEFRAME_M30, '60': mt5.TIMEFRAME_H1}
timePeriod = '15'

p = 5
pips = 0.0001*p

n1 = 8
n2 = 13

In [42]:
def plot_line(ax, ob):
    x, y = ob.xy
    ax.plot(x, y, alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)

In [44]:
from shapely.geometry import LineString

sigNames = ['GBPUSD']
for sig in sigNames:
    print(sig)
    
    for key, val in tqdm(timeDict.items()):
        print(key)

        resultDict[key] = defaultdict(list)

        numYears = 1
        numBefore = int(60/int(key) * 24 * 365 * numYears) # Data for one year 
        print(numBefore)
        
        rates = getData(numBefore, key, sig, timeDict) 
        filteredDf = getAllInfo(rates, n1, n2)
        print(mt5.last_error())       
        
        ## CHECK IF THERE IS A WIN AFTER DOING A CROSS UP
        for index, row in tqdm(filteredDf.iterrows()):
            if index > len(filteredDf)-max(barsAfter)-1:
                 continue
    
            prevBar = filteredDf.iloc[index-1]
            
            # need to check if two lines intersect one another
            l1 = LineString([(0, prevBar[f'ema{n1}']), (1, row[f'ema{n1}'])])
            l2 = LineString([(0, prevBar[f'ema{n2}']), (1, row[f'ema{n2}'])])

            if l1.intersects(l2):
#                 print(pd.to_datetime(row.time, unit='s'))

#                 fig = plt.figure()
#                 ax = fig.add_subplot(111)
                
#                 plot_line(ax, l1)
#                 plot_line(ax, l2)
#                 plt.show()

                # Check up trend
                if row[f'ema{n1}'] > row[f'ema{n2}']:
                
                    for n in barsAfter:
                        nextBars = filteredDf.iloc[index+1:index+n+1]

                        if max(nextBars.high) > (row.close + pips):
                            resultDict[key][n].append(True)
                        else:
                            resultDict[key][n].append(False)
                            
                else:
                    for n in barsAfter:
                        nextBars = filteredDf.iloc[index+1:index+n+1]

                        if min(nextBars.low) < (row.close - pips):
                            resultDict[key][n].append(True)
                        else:
                            resultDict[key][n].append(False)

#     print('timeframe', 'bars_after', 'truth_values', 'percentage_true')
    for key, val in resultDict.items():
        for i, j in val.items():
            n, c = np.unique(j, return_counts=True)
            print(key, i, n, c, c[1]/sum(c)*100)
        print('\n')

GBPUSD


15
35040
(1, 'Success')



30
17520
(1, 'Success')



60
8760
(1, 'Success')




15 3 [False  True] [1006 1177] 53.916628492899676
15 5 [False  True] [ 844 1339] 61.33760879523591
15 10 [False  True] [ 611 1572] 72.01099404489236


30 3 [False  True] [338 709] 67.71728748806113
30 5 [False  True] [256 791] 75.54918815663801
30 10 [False  True] [174 873] 83.3810888252149


60 3 [False  True] [ 80 404] 83.47107438016529
60 5 [False  True] [ 59 425] 87.80991735537191
60 10 [False  True] [ 36 448] 92.56198347107438


